Testing: an esoteric language called "atvian" (hint: latvian) where the position of instructions or variables are semantically significant.

In [3]:
def parse_grid(input_code):
    # Split the input into lines
    lines = input_code.splitlines()
    
    # Convert lines into a grid of tokens with indentation levels
    grid = []
    for line in lines:
        stripped_line = line.lstrip()  # Remove leading spaces
        indent_level = len(line) - len(stripped_line)  # Count leading spaces
        tokens = stripped_line.split()  # Split by spaces into tokens
        grid.append((indent_level, tokens))
    
    return grid

def build_hierarchy(grid):
    hierarchy = []
    stack = []

    for indent, tokens in grid:
        node = {"indent": indent, "tokens": tokens, "children": []}
        
        # If stack is empty, this is the root node
        if not stack:
            hierarchy.append(node)
            stack.append(node)
        else:
            # Find the correct parent based on indentation
            while stack and stack[-1]["indent"] >= indent:
                stack.pop()
            if stack:
                stack[-1]["children"].append(node)
            else:
                hierarchy.append(node)
            stack.append(node)

    return hierarchy


x = """
double
  x
    x * 2
add
  a b
    a + b
"""

build_hierarchy(parse_grid(x))


[{'indent': 0, 'tokens': [], 'children': []},
 {'indent': 0,
  'tokens': ['double'],
  'children': [{'indent': 2,
    'tokens': ['x'],
    'children': [{'indent': 4, 'tokens': ['x', '*', '2'], 'children': []}]}]},
 {'indent': 0,
  'tokens': ['add'],
  'children': [{'indent': 2,
    'tokens': ['a', 'b'],
    'children': [{'indent': 4, 'tokens': ['a', '+', 'b'], 'children': []}]}]}]

In [23]:
class ASTNode:
    def __init__(self, type, value=None, children=None):
        self.type = type  # e.g., "function", "expression", "literal"
        self.value = value  # Name, operator, or literal value
        self.children = children or []  # Child nodes

    def __repr__(self):
        return f"{self.type}({self.value}, {self.children})"


def build_ast(hierarchy):
    def process_node(node):
        tokens = node["tokens"]

        # Function definition
        if len(tokens) == 1 and node["children"]:
            return ASTNode(
                type="function",
                value=tokens[0],
                children=[process_node(child) for child in node["children"]]
            )

        # Argument or literal
        elif len(tokens) == 1:
            try:
                # Try to parse as a number
                return ASTNode(type="literal", value=int(tokens[0]))
            except ValueError:
                # Otherwise, treat as an argument
                return ASTNode(type="argument", value=tokens[0])

        # Expression (e.g., x * 2)
        elif len(tokens) > 1:
            operator = tokens[1]  # Assume the second token is the operator
            operands = [ASTNode(type="argument", value=tokens[0])]
            for token in tokens[2:]:
                try:
                    # Parse as literal if possible
                    operands.append(ASTNode(type="literal", value=int(token)))
                except ValueError:
                    operands.append(ASTNode(type="argument", value=token))
            return ASTNode(type="expression", value=operator, children=operands)

    return [process_node(node) for node in hierarchy]

class Interpreter:
    def __init__(self):
        self.env = {}

    def evaluate(self, node):
        """Evaluate an AST node."""
        if node.type == "literal":
            return node.value
        elif node.type == "argument":
            return self.env.get(node.value, None)  # Get value from environment
        elif node.type == "expression":
            return self.evaluate_expression(node)
        elif node.type == "function":
            return self.evaluate_function(node)
        else:
            raise ValueError(f"Unknown node type: {node.type}")

    def evaluate_expression(self, node):
        """Evaluate an expression node."""
        operator = node.value
        operands = node.children
        evaluated_operands = [self.evaluate(operand) for operand in operands]
        
        if operator == "+":
            return sum(evaluated_operands)
        elif operator == "*":
            result = 1
            for operand in evaluated_operands:
                result *= operand
            return result
        # Add more operators as needed (subtraction, division, etc.)
        else:
            raise ValueError(f"Unknown operator: {operator}")

    def evaluate_function(self, node):
        """Evaluate a function node (define the function)."""
        func_name = node.value
        args = node.children[0].value  # Assume function takes arguments
        print("a")
        print(node.children[0])
        return
        
        body = node.children[1]  # Function body

        self.env[func_name] = {"args": args, "body": body}

    def apply_function(self, func_name, arguments):
        """Apply a function to arguments."""
        func = self.env.get(func_name)
        if not func:
            raise ValueError(f"Function {func_name} not found")
        
        # Bind arguments in the function environment
        local_env = self.env.copy()
        for arg, value in zip(func["args"], arguments):
            local_env[arg] = value
        
        # Evaluate the function body in the local environment
        return self.evaluate(func["body"])


# Define the program

x = """double
  x
    x * 2"""
hierarchy = build_hierarchy(parse_grid(x))

print(hierarchy)
# Build the AST
ast = build_ast(hierarchy)

# Instantiate the interpreter
interpreter = Interpreter()

print(ast)
# First, evaluate the function definition
for node in ast:
    interpreter.evaluate(node)

# Now, apply the function with an argument (e.g., x = 5)
result = interpreter.apply_function("double", [5])

print(result)  # Expected output: 10 (since 5 * 2 = 10)


[{'indent': 0, 'tokens': ['double'], 'children': [{'indent': 2, 'tokens': ['x'], 'children': [{'indent': 4, 'tokens': ['x', '*', '2'], 'children': []}]}]}]
[function(double, [function(x, [expression(*, [argument(x, []), literal(2, [])])])])]
a
function(x, [expression(*, [argument(x, []), literal(2, [])])])


ValueError: Function double not found